In [20]:
import numpy as np
import pandas as pd
import glob
import math 
import os
import lvm_read
import pickle
from datetime import datetime, timedelta
import xarray as xr

In [21]:
#user inputs
#Absolute directory of the folders containing LVM and GPS Data
lvmDirectory = "C:\\Github Repositories\\LabView-Garmin-To-NETCDF\\Test Files 2\\LVM Files"
gpsDirectory = "C:\\Github Repositories\\LabView-Garmin-To-NETCDF\\Test Files 2\\GPS Files"

#convert fit files from GPS to CSVs using the following website:
#https://www.fitfileviewer.com/

In [22]:
#obtain and store GPS data
#https://www.geeksforgeeks.org/how-to-merge-multiple-csv-files-into-a-single-pandas-dataframe/
#merge the files
joined_files = os.path.join(gpsDirectory, "*.csv")

#list of all joined files returned
joined_list = glob.glob(joined_files)

#All files joined to get nontimestamp converted GPS Data
gpsData = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

#Garmin has GPS data starting at a time of Sunday, Dec 31st, 1989. To convert to a time that is useable, first convert to epoch time by adding 631065600 seconds. Then convert epoch time to now time.
#http://www.nlvocables.com/blog/?p=969

gpsData.timestamp = gpsData.timestamp.astype(int) + 631065600

#convert to regular date time
gpsData['timestamp'] = pd.to_datetime(gpsData['timestamp'], unit = 's')
gpsData

#note, for whatever reason, the first file: "2022-10-19 9.56.49_record_1.csv" displays the wrong date for some reason I don't understand. The forumla works for every other file

,timestamp,position_lat,position_long,enhanced_speed,enhanced_altitude
0,2022-10-17 23:09:35,568495601.0,-1.459247e+09,1.53,37.4
1,2022-10-17 23:09:42,568492859.0,-1.459249e+09,1.10,37.4
2,2022-10-17 23:10:00,568487274.0,-1.459250e+09,0.47,37.4
3,2022-10-17 23:10:27,568485049.0,-1.459249e+09,0.41,37.4
4,2022-10-17 23:10:48,568483384.0,-1.459250e+09,0.11,37.4
...,...,...,...,...,...
1374,2022-10-28 17:42:56,574139164.0,-1.461428e+09,2.18,37.4
1375,2022-10-28 17:43:19,574140305.0,-1.461424e+09,0.68,37.4
1376,2022-10-28 17:43:44,574139946.0,-1.461424e+09,1.06,37.4
1377,2022-10-28 17:43:46,574139774.0,-1.461424e+09,1.10,37.4


In [ ]:
#obtain lvm data
#code from this: https://github.com/openmodal/lvm_read/blob/master/Showcase%20lvm_read.ipynb
os.chdir("C:\\Github Repositories\\LabView-Garmin-To-NETCDF")
LVMfilename = 'test_22-10-11_1349_011.lvm'
lvm = lvm_read.read(LVMfilename,read_from_pickle=False)

In [ ]:
#organize file name
#obtain start date
startDate = lvm[0]['Date'][0]
date = startDate.replace('/','-')

#obtain start time
startTime = lvm[0]['Time'][0]

#obtain time values
timeTuple = startTime.split(':',2)

#make file names
fileName = date + '_' + timeTuple[0] + timeTuple[1] + '.nc'

In [ ]:
#calculate time intervals
deltaX = lvm[0]['Delta_X'][0]
samples = lvm[0]['Samples'][0]

timeFloat = round(float(timeTuple[2]),4)

timeArray = [timeFloat]
count = 1

minute = timeTuple[1]

for i in range (0,samples-1):
    nextTime = timeArray[i] + deltaX
    count = count + 1

    if nextTime >= 60:
        break
    else:
        timeArray.append(nextTime)

In [ ]:
#grab Data & convert to pandas dataframe
voltages = []
for i in range (0, count):
    voltage = lvm[0]['data'][i][0]
    voltages.append(voltage)

d = {'Time': timeArray, 'Voltage': voltages}
df = pd.DataFrame(d)

print(df)

In [ ]:
# convert dataframe to an xarray
dataXR = df.to_xarray()

# set variable attribute metadata
dataXR['Time'].attrs = {'units':'seconds'}
dataXR['Voltage'].attrs = {'units':'Volts'}

#global attribute metadata
dataXR.attrs = {'Channels': lvm[0]['Channels'],
            'Samples': lvm[0]['Samples'][0],
            'Date': lvm[0]['Date'][0],
            'Time': lvm[0]['Time'][0],
            'Y_Unit_Label': lvm[0]['Y_Unit_Label'][0],
            'X_Dimension': lvm[0]['X_Dimension'][0],
            'X0': lvm[0]['X0'][0],
            'Delta_X': lvm[0]['Delta_X'][0]}

print(dataXR)

In [ ]:
#convert to netCDF file
dataXR.to_netcdf(fileName)

In [ ]:
#check read netCDF4
ds = xr.open_dataset('2022-10-11_1349.nc')
ds